In [1]:
# https://github.com/anjesh/Jupyter-GeoJson-Mapping
# https://www.ryanbaumann.com/blog/2016/4/3/embedding-mapbox-plots-in
#https://github.com/jwass/geojsonio.py-jupyter-notebooks
# http://jupyter-gmaps.readthedocs.io/en/latest/gmaps.html
#https://github.com/planetlabs/planet-client-python

In [4]:
from planet import api
from planet.api import filters
from sys import stdout
import pandas as pd
import sys
import os
import json
import urllib
import numpy as np
import seaborn as sns
import rasterio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from scipy import ndimage

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
from traitlets import link

from IPython.display import display, Image, HTML
%matplotlib inline

# will pick up api_key via environment variable PL_API_KEY
# but can be specified using `api_key` named argument
api_keys = json.load(open("apikeys.json",'r'))
client = api.ClientV1(api_key=api_keys["PLANET_API_KEY"])


In [11]:
# Planet Basemaps Setup
# Basemap Mosaic (v1 API)
mosaicsSeries = 'global_quarterly_2017q1_mosaic'
# Planet tile server base URL (Planet Explorer Mosaics Tiles)
mosaicsTilesURL_base = 'https://tiles0.planet.com/experimental/mosaics/planet-tiles/' + mosaicsSeries + '/gmap/{z}/{x}/{y}.png'
# Planet tile server url
mosaicsTilesURL = mosaicsTilesURL_base + '?api_key=' + api_keys["PLANET_API_KEY"]
# Map Settings 
# Define colors
colors = {'blue': "#009da5"}
# Define initial map center lat/long
center = [45.5231, -122.6765]
# Define initial map zoom level
zoom = 11
# Set Map Tiles URL
planetMapTiles = TileLayer(url= mosaicsTilesURL)
# Create the map
m = Map(
    center=center, 
    zoom=zoom,
    default_tiles = planetMapTiles # Uncomment to use Planet.com basemap
)
# Define the draw tool type options
polygon = {'shapeOptions': {'color': colors['blue']}}
rectangle = {'shapeOptions': {'color': colors['blue']}} 

# Create the draw controls
# @see https://github.com/ellisonbg/ipyleaflet/blob/master/ipyleaflet/leaflet.py#L293
dc = DrawControl(
    polygon = polygon,
    rectangle = rectangle
)
# Initialize an action counter variable
actionCount = 0
AOIs = {}

# Register the draw controls handler
def handle_draw(self, action, geo_json):
    # Increment the action counter
    global actionCount
    actionCount += 1
    # Remove the `style` property from the GeoJSON
    geo_json['properties'] = {}
    # Convert geo_json output to a string and prettify (indent & replace ' with ")
    geojsonStr = json.dumps(geo_json, indent=2).replace("'", '"')
    AOIs[actionCount] = json.loads(geojsonStr)
    
# Attach the draw handler to the draw controls `on_draw` event
dc.on_draw(handle_draw)
m.add_control(dc)
m

In [12]:
print AOIs[1]
myAOI = AOIs[1]["geometry"]
# geojson AOI
aoi = {

        "type": "Polygon",
        "coordinates": [
          [
            [
              -123.01528930664061,
              45.213003555993964
            ],
            [
              -122.11441040039061,
              45.213003555993964
            ],
            [
              -122.11441040039061,
              45.82401446834977
            ],
            [
              -123.01528930664061,
              45.82401446834977
            ],
            [
              -123.01528930664061,
              45.213003555993964
            ]
          ]
        ]
}


# build a query using the AOI and
# a cloud_cover filter that excludes 'cloud free' scenes
query = filters.and_filter(
    filters.geom_filter(myAOI),
    filters.range_filter('cloud_cover', gt=0),
)

# build a request for only PlanetScope imagery
request = filters.build_search_request(
    query, item_types=['PSScene4Band']
)

# if you don't have an API key configured, this will raise an exception
result = client.quick_search(request)
scenes = []
planet_map = {}
for item in result.items_iter(limit=None):
    planet_map[item['id']]=item
    props = item['properties']
    props["id"] = item['id']
    props["geometry"] = item["geometry"]
    props["thumbnail"] = item["_links"]["thumbnail"]
    scenes.append(props)
scenes = pd.DataFrame(data=scenes)
scenes.head()

{u'geometry': {u'type': u'Polygon', u'coordinates': [[[-122.784513970837, 45.46493701883627], [-122.784513970837, 45.600577826317604], [-122.40411235950887, 45.600577826317604], [-122.40411235950887, 45.46493701883627], [-122.784513970837, 45.46493701883627]]]}, u'type': u'Feature', u'properties': {}}


,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,quality_category,rows,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle
0,2017-04-18T18:13:55.713617Z,0.16,0.61,9065,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.943320,20170418_181355_0f25,PS2,...,test,4650,0f25,471379,135.1,47.7,https://api.planet.com/data/v1/item-types/PSSc...,2017-04-19T13:26:10Z,0,0.7
1,2017-04-18T18:13:54.657796Z,0.32,0.59,9075,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.942415,20170418_181354_0f25,PS2,...,test,4651,0f25,471379,135.2,47.7,https://api.planet.com/data/v1/item-types/PSSc...,2017-04-19T13:26:10Z,0,0.7
2,2017-04-18T18:13:53.601975Z,0.15,0.46,9065,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.942612,20170418_181353_0f25,PS2,...,test,4649,0f25,471379,135.3,47.6,https://api.planet.com/data/v1/item-types/PSSc...,2017-04-19T13:26:10Z,0,0.7
3,2017-04-18T18:13:52.546154Z,0.17,0.47,9079,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.942577,20170418_181352_0f25,PS2,...,standard,4648,0f25,471379,135.4,47.6,https://api.planet.com/data/v1/item-types/PSSc...,2017-04-19T13:26:09Z,0,0.7
4,2017-04-18T00:30:21.952098Z,0.00,0.16,4802,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.977765,20170418_003021_1_0c75,PS2,...,standard,6709,0c75,470582,259.5,25.2,https://api.planet.com/data/v1/item-types/PSSc...,2017-04-18T20:48:22Z,0,1.4


In [13]:
# now let's clean up the datetime stuff
scenes["acquired"] = pd.to_datetime(scenes["acquired"])
scenes["published"] = pd.to_datetime(scenes["published"])
scenes["updated"] = pd.to_datetime(scenes["updated"])

In [14]:
import datetime 
# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.1
good = scenes['quality_category']=="standard"
recent = scenes["acquired"] > datetime.date(year=2017,month=4,day=15)
good_scenes = scenes[(good&clear&recent)]
good_scenes

,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,quality_category,rows,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle
24,2017-04-15 20:18:34.003587,0.01,0.01,4895,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.923194,20170415_201834_1_0d06,PS2,...,standard,6562,0d06,467731,183.8,54.5,https://api.planet.com/data/v1/item-types/PSSc...,2017-04-16 03:26:47,0,0.9
26,2017-04-15 20:18:30.991587,0.01,0.01,4893,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.923703,20170415_201830_1_0d06,PS2,...,standard,6561,0d06,467731,183.4,54.6,https://api.planet.com/data/v1/item-types/PSSc...,2017-04-16 03:26:47,0,0.8
27,2017-04-15 20:18:30.238587,0.01,0.01,4891,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.921326,20170415_201830_0d06,PS2,...,standard,6550,0d06,467731,183.3,54.6,https://api.planet.com/data/v1/item-types/PSSc...,2017-04-16 03:26:47,0,0.9


In [15]:
imgs = []
for img in good_scenes["thumbnail"].tolist():
    imgs.append(Image(url=img))
display(*imgs)

In [16]:
def get_products(scene_id,asset_type='PSScene4Band'):    
    out = client.get_assets_by_id(asset_type,scene_id)
    temp = out.get()
    return temp.keys()


def activate_product(scene_id,asset_type="PSScene4Band",product="analytic"):
    temp = client.get_assets_by_id(asset_type,scene_id)  
    products = temp.get()
    if( product in products.keys() ):
        return client.activate(products[product]),products[product]
    else:
        return None 
    
to_get = good_scenes["id"].tolist()
print get_products(to_get[0])
thing,product = activate_product(to_get[0])
derp = client.download(product)
        
#to_get = good_scenes["id"].tolist()
#out = client.get_assets_by_id('PSScene4Band',to_get[0])
#temp = out.get()
#for k,v in temp.items():
#    print k
#print out.get()
#client.activate(temp["analytic"])
#print type(out.get())
#temp = json.loads(out.get())
#print temp
#client.activate(json.loads(out.get()))

[u'basic_analytic_rpc_nitf', u'analytic_xml', u'basic_analytic_dn', u'basic_analytic_dn_xml_nitf', u'basic_analytic_dn_nitf', u'basic_analytic_xml', u'basic_analytic_nitf', u'basic_analytic_rpc', u'analytic_dn', u'basic_udm', u'basic_analytic_dn_rpc_nitf', u'analytic', u'analytic_dn_xml', u'basic_analytic_dn_xml', u'basic_analytic_dn_rpc', u'basic_analytic_xml_nitf', u'basic_analytic', u'udm']


In [17]:
derp.

SyntaxError: invalid syntax (<ipython-input-17-5467e061cb6d>, line 1)

In [ ]:
import planet as pl

In [ ]:
pl.api.ClientV1.activate?
